In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


# 1. Scouter

In [10]:
%%time
from modules.scouter_handler import ScouterHandler
from modules.scouter.config import default_addr, default_newspaper_idx, default_timeline_idx
from modules.topic_generator import TopicGenerator

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.2 µs


In [11]:
keyword = "삼성전자"
out_path = "results/"

In [12]:
scouter = ScouterHandler(addr=default_addr, size=1000)
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'extContent'])
doc_info_list = scouter.search(query_body, data_type=default_newspaper_idx, max_num_doc=2000, trim_lower=False)

Query : 삼성전자
Scroll idx : 1 (1000 docs)
Scroll idx : 2 (1000 docs)
Total retrieved Doc # :  2000



In [13]:
date = '2019-05-08'
topic_gen = TopicGenerator(keyword, out_path)
topic_doc = topic_gen.process_data(date)
clusters = topic_doc['clusters']

for i in range(len(clusters)):
    print('Topic {} keywords :'.format(i+1), clusters[i]['keywords'])
    print('Topic {} news article & # of doc {}:'.format(i+1, len(clusters[i]['news_source'])))
    print(clusters[i]['news_source'], end='\n\n')

Topic 1 keywords : ['수출', '부진', '경상흑자']
Topic 1 news article & # of doc 9:
[{'news_id': 6548063, 'postingDate': '2019-05-08', 'newsTitle': '불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저', 'companyName': '디지털타임스', 'category': '금융', 'extContent': '올 1분기 우리나라의 경상수지 흑자 규모가 6년여 만에 가장 적은 수준으로 축소됐다. 1분기 경상수지 흑자 규모는 112억5000만달러로 27분기만에 가장 적었다. 반도체와 석유류 등 수출 부진으로 상품수지 흑자 규모가 크게 둔화된 결과다. 8일 한국은행이 발표한  2019년 3월 및 1분기 국제수지(잠정) 에 따르면 1분기 경상수지는 112억5000만달러 흑자로 지난 2012년 2분기(109억4000만달러 흑자) 이후 6년9개월(27분기)만에 최소치를 기록했다. 이는 상품수지가 196억1000만달러로 2014년 1분기(170억6000만달러)이후 20분기 만에 최소 흑자를 낸 데 따른 영향이다. 1분기 수출(1375억달러)과 수입(1178억9000만달러)은 각각 2016년 3분기 이후 10분기 만에 동반 감소했다. 1분기 수출은 1375억달러로 1년 전보다 8.4% 줄었다. 분기별 수출이 감소한 것은 2016년 3분기(-3.9%) 이후 2년 6개월 만이다. 한은은 세계 교역량 둔화, 반도와체 석유류 수출 감소, 중국 수출 부진 등을 원인으로 꼽았다. 수입 또한 기계 등 자본재 수입 감소, 원유도입단가 하락 전환 등으로 감소했다. 1분기 수입이 1178억9000만달러로 1년 전보다 7.6% 감소하면서 상품수지 흑자를 유지했지만, 수출과 수입이 동반 감소하는 불황형 흑자 양상을 보였다. 다만 서비스수지와 이전소득수지가 개선되며 경상수지를 떠받쳤다. 서비스수지 적자 규모는 여행과 운송수지 개선으로 76억6000만 달러를 기록했다. 이는 지난해 4분기(-63억3

# 2. Multi-document Summary

In [15]:
%%time
from modules.multi_doc_summary import MultiDocSummary

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py", line 1246, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
  File "/workspace/wise_reporter/modules/multi_doc_summary.py", line 4, in <module>
    from modules.multi_summ.multi_doc_sum import mds
  File "/workspace/wise_reporter/modules/multi_summ/multi_doc_sum.py", line 3, in <module>
    from onmt.utils.logging import init_logger
ModuleNotFoundError: No module named 'onmt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.

ModuleNotFoundError: No module named 'onmt'

# 3. TimelineSummary

In [ ]:
from modules.timeline_summary import TimelineSummary

In [ ]:
%%time
multi_doc_summ = MultiDocSummary(keyword, out_path)
multi_doc_summ.process_data(top_doc_morph_sentence_list, top_keyword_list)

In [9]:

from modules.image_retrieve import ImageRetriever

from modules.utils import make_morp_sentence_list

Query : 미투
Scroll idx : 1 (813 docs)
Trim lower 70% of docs (243 docs are cut)
Total retrieved Doc # :  569

CPU times: user 1.01 s, sys: 212 ms, total: 1.22 s
Wall time: 1.87 s


In [10]:
%%time
time_line_summ = TimelineSummary(keyword, out_path)
time_line_summ.process_data(doc_info_list)

A :  0.010506391525268555
B :  0.9801366329193115
C :  0.0604701042175293
len(files) :  190
len(sents) :  15
len(sents) :  10
len(sents) :  19
len(sents) :  12
len(sents) :  12
len(sents) :  14
len(sents) :  13
len(sents) :  10
len(sents) :  16
len(sents) :  7
len(sents) :  10
len(sents) :  6
len(sents) :  7
len(sents) :  5
len(sents) :  1
len(sents) :  3
len(sents) :  13
len(sents) :  5
len(sents) :  2
len(sents) :  1
len(sents) :  1
len(sents) :  1
C-sub :  0.6147806644439697
D :  0.02740788459777832
CPU times: user 1.69 s, sys: 20 ms, total: 1.71 s
Wall time: 1.7 s


In [14]:
%%time
image_ret = ImageRetriever(keyword, out_path, from_google=True)
a = image_ret.process_data(doc_graph_anal.top_keyword_list)


Item no.: 1 --> Item name = 비트코인 정부 규제 금융 은행 미국
Item no.: 1 --> Item name = 비트코인 비트 가능 비트코인 수익 몰수
Evaluating...

Item no.: 1 --> Item name = 비트코인 방송 압박 지지 조사 올림픽

Item no.: 1 --> Item name = 비트코인 기술 블록체 세계 블록 체인

Item no.: 1 --> Item name = 비트코인 장관 법무 폐쇄 발언 부처
Item no.: 1 --> Item name = 비트코인 경찰 설명 이용 채굴 자금Evaluating...

Evaluating...

Evaluating...

Item no.: 1 --> Item name = 비트코인 결제 업체 확인 시스템 사용
Evaluating...


Item no.: 1 --> Item name = 비트코인 투자 가격 시장 지난 시세
Evaluating...
Evaluating...
Evaluating...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Starting Download...
Completed Image ====> 1. f6844a27e3a3c9fb40e77d6ea0db6946.jpg
Completed Image ====> 1. ps17092400316.jpg
Completed Image ====> 2. 2018020201011_0.jpg
Completed Image ====> 1. fd8d13e3-2489-472c-950a-fe804622f910.jpg
Starting Download...
Completed Image ====> 1. image_2112881331525605696709.jpg
Completed Image ====> 1. 01.15405772.1.jpg
Comple

Completed Image ====> 20. nisi20160912_0012171017_web.jpg

Errors: 0

Completed Image ====> 19. image_readtop_2018_696320_15415380623522327.jpg
can not fine image - continue to next image
Completed Image ====> 9. pro-bitcoin-ohio-bill-promotes-government-adoption-of-blockchain-ccn.jpg
can not fine image - continue to next image
Completed Image ====> 10. 2019020701435838.jpg
Completed Image ====> 11. 1499661599-35.jpg
Completed Image ====> 20. ssi_20180111174120_v.jpg

Errors: 1

Completed Image ====> 16. 0128154862889520190135.jpg
Completed Image ====> 12. coinclip_thread_8459062.7028189231550127853612.png
Completed Image ====> 17. 201712050480295258_1.jpg
can not fine image - continue to next image
Completed Image ====> 13. %ed%81%ac%eb%a1%9c%ec%8a%a4%ec%b2%b4%ec%9d%b8%ea%b8%b0%ec%88%a0%ea%b0%9c%eb%85%90%eb%8f%84.jpg
can not fine image - continue to next image
Completed Image ====> 17. %eb%a9%94%ec%9d%b8.jpg
Completed Image ====> 14. 1981902326_1456207779.42026.jpg
Completed Image ===

IndexError: list index out of range